In [1]:
# Only execute if running in container
# !apt install libsm6 libxext6 libxrender-dev git -y
# !pip3 install pandas scikit-learn opencv-python tqdm

In [1]:
import argparse
import os
import pickle
import sys
from collections import Counter
from datetime import datetime

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras.layers import (Activation, Conv2D, Dense, Dropout, Flatten,
                          MaxPooling2D)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import Sequence

In [2]:
print(tf.__version__)
print("GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.0.0
GPUs Available:  1


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--vidpath', default='vids/scaled')
parser.add_argument('--epochs', default=10, type=int)
parser.add_argument('--batch_size', default=32, type=int)
try:
    args = parser.parse_args()
except:
    # may need to modify vidpath depending on where you are running the script
#     args = parser.parse_args(['--vidpath=/tf/data/vids/scaled', '--batch_size=4'])
    args = parser.parse_args(['--vidpath=/data/vids/scaled'])
print(args.vidpath, args.epochs)

/data/vids/scaled 10


usage: ipykernel_launcher.py [-h] [--vidpath VIDPATH] [--epochs EPOCHS]
                             [--batch_size BATCH_SIZE]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-fe314631-0871-4cbf-b7f0-af8e4f517d54.json


In [4]:
# labelPath = 'bittah-ninja/first_1k_labeled_long_vids_removed.csv'
labelPath = 'week10_labeled.csv'
# labelPath = 'full_labels.csv'
df = pd.read_csv(labelPath)
df.head()

,video-UID,clip_title,labeler,class
0,10000,5yo_vs_Mickey_slice0.mp4_slice0.mp4,NaN,99
1,10001,5yo_vs_Mickey_slice10.mp4_slice0.mp4,NaN,99
2,10002,5yo_vs_Mickey_slice15.mp4_slice0.mp4,NaN,99
3,10003,5yo_vs_Mickey_slice20.mp4,Ahsen,0
4,10004,5yo_vs_Mickey_slice5.mp4,Jen,0


In [5]:
new_files = []
for file in df.clip_title:
    newfile = ''.join(file.split('.mp4')) + '.mp4'
    new_files.append(newfile)
df['clip_title'] = new_files
df['label'] = df['class']
df.drop(columns=['class'], inplace=True)
df.groupby('label').size()

label
-1       21
 0     1259
 1      319
 2       14
 3       84
 4       52
 99    3692
dtype: int64

In [6]:
df = df.loc[df.label != -1]
df = df.loc[df.label != 99]
df.groupby('label').size()

label
0    1259
1     319
2      14
3      84
4      52
dtype: int64

In [7]:
df.shape

(1728, 4)

In [8]:
# df['punch'] = (df.label != 0).astype('int')
# df.groupby('punch').size()


In [9]:
vidPath = args.vidpath
filenames = [f.split('.mp4')[0] + '_scaled.mp4' for f in df.clip_title]
filenames = [os.path.join(vidPath, f) for f in filenames]
# labels = df.punch.tolist()
labels = df.label.tolist()


In [13]:
# !ls -l /tf/data/vids/scaled | grep Australian
!ls -l /data/vids/scaled | grep Australian

-rw-r--r-- 1 root root  64160 Nov  9 17:05 Australian_flash_mob_dance_slice0_scaled.mp4
-rw-r--r-- 1 root root 281614 Nov  9 17:05 Australian_flash_mob_dance_slice100_scaled.mp4
-rw-r--r-- 1 root root 315922 Nov  9 17:05 Australian_flash_mob_dance_slice105_scaled.mp4
-rw-r--r-- 1 root root 196418 Nov  9 17:05 Australian_flash_mob_dance_slice10_scaled.mp4
-rw-r--r-- 1 root root 309405 Nov  9 17:05 Australian_flash_mob_dance_slice110_scaled.mp4
-rw-r--r-- 1 root root 280338 Nov  9 17:05 Australian_flash_mob_dance_slice115_scaled.mp4
-rw-r--r-- 1 root root 271860 Nov  9 17:05 Australian_flash_mob_dance_slice120_scaled.mp4
-rw-r--r-- 1 root root 292520 Nov  9 17:05 Australian_flash_mob_dance_slice125_scaled.mp4
-rw-r--r-- 1 root root 291540 Nov  9 17:05 Australian_flash_mob_dance_slice130_scaled.mp4
-rw-r--r-- 1 root root 289584 Nov  9 17:05 Australian_flash_mob_dance_slice135_scaled.mp4
-rw-r--r-- 1 root root 229423 Nov  9 17:05 Australian_flash_mob_dance_slice140_scaled.mp4
-rw-r--r-- 1 

In [14]:
class DataGenerator(Sequence):

    def __init__(self,
                 filenames,
                 labels,
                 batch_size,
                 frame_height=224,
                 frame_width=224,
                 n_channels=1):
        self.filenames = filenames
        self.labels = labels
        self.batch_size = batch_size
        self.h = frame_height
        self.w = frame_width
        self.n_channels = n_channels

    def __len__(self):
        return np.floor(len(self.filenames) / self.batch_size).astype(int)

    def __data_generation(self, idx_list):
        def getSingleFrame(filepath):
            cap = cv2.VideoCapture(filepath)
            vid = []
            while cap.isOpened():
                ret, frame = cap.read()
                if ret:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    gray = cv2.resize(gray, (self.h, self.w))
                    vid.append(gray)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                else:
                    break
            cap.release()
            j = int(np.random.choice(len(vid), 1))
            frame = vid[j]
            return frame

        x = np.empty((self.batch_size,
                      self.w,
                      self.h,
                      self.n_channels), dtype=np.float16)
        y = np.empty(self.batch_size, dtype=np.float16)
        for i, idx in enumerate(idx_list):
#             print(idx)
            file = self.filenames[idx]
            frame = getSingleFrame(file)
            frame = frame.reshape(self.w,
                                  self.h,
                                  self.n_channels)
            x[i, ] = frame
            y[i, ] = self.labels[idx]
        y = tf.keras.utils.to_categorical(
            y, num_classes=len(set(self.labels)), dtype='float16')
        return x, y
        # yield x, y

    def __getitem__(self, idx):
        batch = range(idx * self.batch_size, (idx + 1) * self.batch_size)
        x, y = self.__data_generation(batch)
#         print(f'\n{idx} {x.shape} {y.shape}')
        return x, y
        # yield x, y



In [15]:
# def __data_generation(idx_list):
#     def getSingleFrame(filepath):
#         cap = cv2.VideoCapture(filepath)
#         vid = []
#         while cap.isOpened():
#             ret, frame = cap.read()
#             if ret:
#                 gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#                 gray = cv2.resize(gray, (224,224))
#                 vid.append(gray)
#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                     break
#             else:
#                 break
#         cap.release()
#         j = int(np.random.choice(len(vid), 1))
#         frame = vid[j]
#         return frame

#     x = np.empty((args.batch_size,
#                   224,
#                   224,
#                   1), dtype=np.float16)
#     y = np.empty(args.batch_size, dtype=np.float16)
#     print(x.shape, y.shape)
#     for i, idx in enumerate(idx_list):
#         file = filenames[idx]
#         frame = getSingleFrame(file)
#         frame = frame.reshape(224,
#                               224,
#                               1)
#         x[i, ] = frame
#         y[i, ] = labels[idx]
#         print(x.shape, y.shape)
#     y = tf.keras.utils.to_categorical(
#         y, num_classes=len(set(labels)), dtype='float16')
#     print(x.shape, y.shape)
#     return x, y

In [16]:
# x, y = __data_generation(range(args.batch_size))

In [17]:
# np.array(list(set(labels)))
np.unique(labels)

array([0, 1, 2, 3, 4])

In [18]:
batch_size = args.batch_size
class_weight = compute_class_weight('balanced', np.unique(labels), labels)
# labels_counts = Counter(labels)
# # TODO: Not sure if this should be 1 - x or x
# class_weight = {k:1-(v/len(labels)) for k,v in labels_counts.items()}

In [19]:
class_weight

array([ 0.27450357,  1.08338558, 24.68571429,  4.11428571,  6.64615385])

In [20]:
x_train, x_test, y_train, y_test = train_test_split(
    filenames, labels, test_size=0.2)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

1382 1382
346 346


In [21]:
# max_frame_count = getMaxFrameCount(filenames)
train_generator = DataGenerator(x_train,
                                y_train,
                                batch_size)
test_generator = DataGenerator(x_test,
                               y_test,
                               batch_size)
len(train_generator), len(test_generator)

(43, 10)

In [22]:
batch_size

32

In [23]:
input_shape = (224, 224, 1)
epochs = args.epochs
inputs = keras.layers.Input(shape=input_shape, name='inputs')
conv = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(32, (3, 3), activation='relu', padding='same')(pool)
pool = MaxPooling2D(pool_size=(2, 2))(conv)
# dropout = Dropout(0.25)(pool)
conv = Conv2D(32, (4, 4), activation='relu', padding='same')(pool)
pool = MaxPooling2D(pool_size=(2, 2))(conv)
conv = Conv2D(32, (4, 4), activation='relu', padding='same')(pool)
pool = MaxPooling2D(pool_size=(2, 2))(conv)
dropout = Dropout(0.5)(pool)
flat = Flatten()(dropout)
dense = Dense(16, activation='relu')(flat)
dropout = Dropout(0.25)(dense)
outputs = Dense(len(set(labels)), activation='softmax', name='outputs')(dropout)
model = Model(inputs, outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 224, 224, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 32)        16416     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 32)        0     

In [24]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
hist = model.fit_generator(generator=train_generator,
                           steps_per_epoch=(len(x_train) // batch_size),
                           epochs=100,
                           verbose=1,
                           validation_data=test_generator,
                           validation_steps=(len(x_test) // batch_size),
                           shuffle=False,
                           class_weight=class_weight,
                           use_multiprocessing=False,
                           callbacks=[es])


Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[32,224,224,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [ ]:
dt = datetime.now().strftime("%Y%m%d_%H%M%S")
modelpath = '/tf/data/models'
os.makedirs(modelpath, exist_ok=True)
model.save(os.path.join(modelpath, f'simpleCNN_{args.epochs}epochs_{dt}.h5'))
with open(os.path.join(modelpath, f'simpleCNN_history_{args.epochs}epochs_{dt}.pickle'), 'wb') as f:
    pickle.dump(hist.history, f, protocol=-1)

In [ ]:
!ls -l /tf/data/models